In [1]:
import pandas as pd #pip install pandas openpyxl

In [2]:
#input data dari explorasi data A
df = pd.read_excel("D:\Semester 7\TA\draft_multiapp\data\data_template.xlsx", 
                       engine='openpyxl',
                       sheet_name='Peg tetap',
                       usecols='A:AK',
                       nrows=2668,
                       )

In [3]:
# tingkat bunga aktuaria
tingkat_bunga = 0.0698
tingkat_bunga

0.0698

In [4]:
# tingkat kenaikan gaji
tingkat_kenaikan_gaji = 0.025
tingkat_kenaikan_gaji

0.025

In [5]:
# replacement ratio
replacement_ratio_max = 0.75
replacement_ratio_min = 0.4
pensiun_normal_ratio = 0.025
masa_kerja_minimal = 20

In [14]:
df["Gapok Terakhir"] = df["Gaji Pokok"] * (1 + tingkat_kenaikan_gaji) ** (df["Usia Pensiun (r) "] - df["Usia sekarang (x)"] - 1)
df["Benefit Pensiun Normal (1 Bulan)"] = 0.025 * df["Total Masa Kerja Harapan (MK)"] * df["Gapok Terakhir"]

condition_1 = df["Total Masa Kerja Harapan (MK)"] >= 20
condition_2 = df["Total Masa Kerja Harapan (MK)"] > replacement_ratio_max / pensiun_normal_ratio
condition_3 = df["Total Masa Kerja Harapan (MK)"] < replacement_ratio_min / pensiun_normal_ratio

df.loc[condition_1 & condition_2, "Benefit Pensiun Normal Taspen (1 Bulan)"] = replacement_ratio_max * df.loc[condition_1 & condition_2, "Gapok Terakhir"]
df.loc[condition_1 & condition_3, "Benefit Pensiun Normal Taspen (1 Bulan)"] = replacement_ratio_min * df.loc[condition_1 & condition_3, "Gapok Terakhir"]
df.loc[(condition_1 & ~(condition_2 | condition_3)), "Benefit Pensiun Normal Taspen (1 Bulan)"] = pensiun_normal_ratio * df.loc[(condition_1 & ~(condition_2 | condition_3)), "Total Masa Kerja Harapan (MK)"] * df.loc[(condition_1 & ~(condition_2 | condition_3)), "Gapok Terakhir"]

df["Benefit Pensiun Normal (1 Tahun)"] = df["Benefit Pensiun Normal (1 Bulan)"] * 12
df["Benefit Pensiun Normal Taspen (1 Tahun)"] = df["Benefit Pensiun Normal Taspen (1 Bulan)"] * 12
df["Replacement Ratio"] = df["Benefit Pensiun Normal Taspen (1 Bulan)"] / df["Gapok Terakhir"]
        

In [15]:
df
#format nilai dalam rupiah
#st.session_state.df["Benefit Pensiun Normal (1 Bulan)"] = st.session_state.df["Benefit Pensiun Normal (1 Bulan)"].map('Rp{:,.2f}'.format)

,No.,staff_id,Jenis Kelamin,Tanggal lahir (DD/MM/YYYY),Tanggal mulai bekerja,IURAN,Hubungan Ahli waris 1,Jenis Kelamin ahli waris 1,Tanggal lahir (DD/MM/YYYY) ahli waris 1,Hubungan Ahli waris 2,...,Usia Anak,delta,MK,Gapok Terakhir,Benefit Pensiun Normal (1 Bulan),Benefit Pensiun Normal (1 Tahun),Benefit Pensiun Normal Taspen(1 Bulan),Benefit Pensiun Normal Taspen (1 Bulan),Benefit Pensiun Normal Taspen (1 Tahun),Replacement Ratio
0,1,56572,M,11/03/1991,01-01-2020,NaN,1.0,F,09/10/1991,4.0,...,5,-27,3,6.111459e+06,5.500313e+06,6.600375e+07,4.583594e+06,4.583594e+06,5.500313e+07,0.750
1,2,42729,M,16/12/1996,01-01-2022,NaN,1.0,F,29/11/1996,4.0,...,3,-24,1,5.362694e+06,5.228627e+06,6.274352e+07,4.022021e+06,4.022021e+06,4.826425e+07,0.750
2,3,17955,M,12/04/1995,01-01-2023,NaN,1.0,F,22/12/1994,3.0,...,2,-26,0,5.231897e+06,4.839505e+06,5.807406e+07,3.923923e+06,3.923923e+06,4.708707e+07,0.750
3,4,13254,M,10/04/1969,01-02-2016,NaN,1.0,F,05/06/1975,3.0,...,0,-54,7,3.936900e+06,1.771605e+06,2.125926e+07,NaN,NaN,NaN,NaN
4,5,56874,M,23/06/1989,01-01-2020,NaN,1.0,F,19/02/1990,NaN,...,,,3,5.878014e+06,4.996312e+06,5.995574e+07,4.408510e+06,4.408510e+06,5.290212e+07,0.750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2663,2664,11513,M,14/09/1983,01-09-2020,NaN,1.0,F,18/05/1997,3.0,...,5,-35,3,3.367728e+06,1.599671e+06,1.919605e+07,NaN,NaN,NaN,NaN
2664,2665,10218,M,29/05/1982,01-11-2016,NaN,1.0,F,03/07/1984,4.0,...,8,-33,7,4.719756e+06,2.595866e+06,3.115039e+07,2.595866e+06,2.595866e+06,3.115039e+07,0.550
2665,2666,10222,F,05/04/1982,01-09-2019,NaN,NaN,NaN,NaN,4.0,...,20,-21,4,3.797791e+06,1.803951e+06,2.164741e+07,NaN,NaN,NaN,NaN
2666,2667,9105,M,02/09/1982,02-01-2013,NaN,1.0,F,22/05/1982,4.0,...,11,-30,10,4.919410e+06,3.074631e+06,3.689557e+07,3.074631e+06,3.074631e+06,3.689557e+07,0.625


In [12]:

# Specify the file path where you want to save the Excel file
#excel_file_path = 'D:\Semester 7\TA\draft_multiapp\data\data_template2.xlsx'

# Export the DataFrame to an Excel file
#df.to_excel(excel_file_path, index=False)

,No.,staff_id,Jenis Kelamin,Tanggal lahir (DD/MM/YYYY),Tanggal mulai bekerja,IURAN,Hubungan Ahli waris 1,Jenis Kelamin ahli waris 1,Tanggal lahir (DD/MM/YYYY) ahli waris 1,Hubungan Ahli waris 2,...,Usia Anak 1,Usia Anak 2,Usia Anak,delta,MK,Gapok Terakhir,Benefit Pensiun Normal (1 Bulan),Benefit Pensiun Normal (1 Tahun),Benefit Pensiun Normal Taspen(1 Bulan),Benefit Pensiun Normal Taspen (1 Bulan)
0,1,56572,M,11/03/1991,01-01-2020,NaN,1.0,F,09/10/1991,4.0,...,5,,5,-27,3,6.111459e+06,5.500313e+06,6.600375e+07,4.583594e+06,4.583594e+06
1,2,42729,M,16/12/1996,01-01-2022,NaN,1.0,F,29/11/1996,4.0,...,3,,3,-24,1,5.362694e+06,5.228627e+06,6.274352e+07,4.022021e+06,4.022021e+06
2,3,17955,M,12/04/1995,01-01-2023,NaN,1.0,F,22/12/1994,3.0,...,2,,2,-26,0,5.231897e+06,4.839505e+06,5.807406e+07,3.923923e+06,3.923923e+06
3,4,13254,M,10/04/1969,01-02-2016,NaN,1.0,F,05/06/1975,3.0,...,NaN,17,0,-54,7,3.936900e+06,1.771605e+06,2.125926e+07,NaN,NaN
4,5,56874,M,23/06/1989,01-01-2020,NaN,1.0,F,19/02/1990,NaN,...,,,,,3,5.878014e+06,4.996312e+06,5.995574e+07,4.408510e+06,4.408510e+06
5,6,18958,F,30/05/1993,01-11-2018,NaN,NaN,NaN,NaN,NaN,...,,,,,5,6.622979e+06,6.622979e+06,7.947575e+07,4.967234e+06,4.967234e+06
6,7,16826,F,04/11/1986,01-06-2017,NaN,NaN,NaN,NaN,NaN,...,,,,,6,5.571682e+06,4.735930e+06,5.683116e+07,4.178761e+06,4.178761e+06
7,8,9310,M,10/01/1978,01-04-2013,NaN,1.0,F,03/11/1979,3.0,...,13,10,10,-35,10,5.018802e+06,3.764102e+06,4.516922e+07,3.764102e+06,3.764102e+06
8,9,71503,M,16/12/1986,01-04-2023,NaN,NaN,NaN,NaN,NaN,...,,,,,0,4.366656e+06,3.056659e+06,3.667991e+07,3.056659e+06,3.056659e+06
9,10,17191,F,08/07/1993,01-02-2018,NaN,NaN,NaN,NaN,NaN,...,,,,,5,6.622979e+06,6.622979e+06,7.947575e+07,4.967234e+06,4.967234e+06
